# Maps

In [5]:
# Import dependencies
import requests
import json
import pandas as pd
import hvplot.pandas
import cartopy
import geoviews as gv
from config import geoapify_key 


In [ ]:
# Import Data and move index back
oakland_data = pd.read_csv("output_data/oakland_data.csv",index_col="datapoint")
kent_data    = pd.read_csv("output_data/kent_data.csv"   ,index_col="datapoint")
saginaw_data = pd.read_csv("output_data/saginaw_data.csv",index_col="datapoint")

oakland_data["Lat"] = 0.1
oakland_data["Lon"] = 0.1
saginaw_data["Lat"] = 0.1
saginaw_data["Lon"] = 0.1
kent_data["Lat"] = 0.1
kent_data["Lon"] = 0.1

In [ ]:
# Set up function for gathering coordinates for zipcodes
start_url = "https://api.geoapify.com/v1/geocode/search?text="
end_url =  "&lang=en&limit=1&type=postcode&filter=countrycode:us&apiKey="+geoapify_key
def get_coords(df):
    lons = [0]
    lats = [0]
    zips = [0]
    for zipcode in df["Zipcode"].unique():
        response = requests.get(start_url+str(zipcode)+end_url).json()
        lon = response["features"][0]["properties"]["lon"]
        lat = response["features"][0]["properties"]["lat"]
        lons.append(lon)
        lats.append(lat)
        zips.append(zipcode)

    temp_df = pd.DataFrame({"Zipcode":zips,"Lon":lons,"Lat":lats})
    for i in df.index:
        zipcode = df.at[i,"Zipcode"]
        zip_loc = temp_df.loc[temp_df["Zipcode"]== zipcode]
        df.at[i,"Lon"] = zip_loc["Lon"]
        df.at[i,"Lat"] = zip_loc["Lat"]

In [ ]:
# Run function for each dataframe to gather the coordinates
get_coords(oakland_data)
get_coords(kent_data)
get_coords(saginaw_data)

In [ ]:
# Export to CSV to save the API
oakland_data.to_csv("output_data/oakland_latlon.csv",index_label = "datapoint")
saginaw_data.to_csv("output_data/saginaw_latlon.csv",index_label = "datapoint")
kent_data.to_csv("output_data/kent_latlon.csv",index_label = "datapoint")

In [6]:
# Import CSVs to continue using data
oakland_final = pd.read_csv("output_data/oakland_latlon.csv",index_col="datapoint")
kent_final = pd.read_csv("output_data/kent_latlon.csv"   ,index_col="datapoint")
saginaw_final = pd.read_csv("output_data/saginaw_latlon.csv",index_col="datapoint")

In [71]:
oakland_final["% Poverty"] = 100*oakland_final["Poverty Count"]/oakland_final["Total Population"]
kent_final["% Poverty"] = 100*kent_final["Poverty Count"]/kent_final["Total Population"]
saginaw_final["% Poverty"] = 100*saginaw_final["Poverty Count"]/saginaw_final["Total Population"]

## Oakland County

In [72]:
def get_graphs(df,var,county,scale,clim):
    for year in range(2011,2021):
        filtered_data = df.loc[df["Year"]==year]
        cur_map = filtered_data.hvplot.points("Lon","Lat",xlabel = "Longitude",ylabel="Latitude", 
                                              frame_width = 700, frame_height = 500, geo = True, tiles = "EsriStreet", 
                                              hover_cols = ["Zipcode",var], title = f"{county} County {var}: {year}", size = var, 
                                              scale = scale,color=var,clim=clim)
        hvplot.save(cur_map,f"output_data/map_plots/{county}_{var}_{year}.png")

In [73]:
# Total Population
get_graphs(oakland_final,"Total Population","Oakland",.2,(0,50000))

In [91]:
# Poverty Count
get_graphs(oakland_final,("% Poverty"),"Oakland",5,(0,25))

In [92]:
# % Unemployed
get_graphs(oakland_final,"% Unemployed","Oakland",5,(0,10))

In [76]:
# Adjusted Median Income
get_graphs(oakland_final,"Adjusted Median Income","Oakland",.1,(0,150000))

In [77]:
# Adjusted Median Rent
get_graphs(oakland_final,"Adjusted Median Rent","Oakland",1,(0,2000))

In [78]:
# Adjusted Median Home Value
get_graphs(oakland_final,"Adjusted Median Home Value","Oakland",.07,(0,750000))

## Kent County

In [79]:
# Total Population
get_graphs(kent_final,"Total Population","Kent",.2,(0,50000))

In [93]:
# Poverty Count
get_graphs(kent_final,("% Poverty"),"Kent",5,(0,25))

In [94]:
# % Unemployed
get_graphs(kent_final,"% Unemployed","Kent",5,(0,10))

In [82]:
# Adjusted Median Income
get_graphs(kent_final,"Adjusted Median Income","Kent",.1,(0,150000))

In [83]:
# Adjusted Median Rent
get_graphs(kent_final,"Adjusted Median Rent","Kent",1,(0,2000))

In [84]:
# Adjusted Median Home Value
get_graphs(kent_final,"Adjusted Median Home Value","Kent",.07,(0,750000))

## Saginaw County

In [85]:
# Total Population
get_graphs(saginaw_final,"Total Population","Saginaw",.2,(0,50000))

In [95]:
# Poverty Count
get_graphs(saginaw_final,("% Poverty"),"Saginaw",5,(0,25))

In [96]:
# % Unemployed
get_graphs(saginaw_final,"% Unemployed","Saginaw",5,(0,10))

In [88]:
# Adjusted Median Income
get_graphs(saginaw_final,"Adjusted Median Income","Saginaw",.1,(0,150000))

In [89]:
# Adjusted Median Rent
get_graphs(saginaw_final,"Adjusted Median Rent","Saginaw",1,(0,2000))

In [90]:
# Adjusted Median Home Value
get_graphs(saginaw_final,"Adjusted Median Home Value","Saginaw",.07,(0,750000))